<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/LLM_GPT2_Entropy_Semantic_Entropy_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install datasets

GPT2_FFT

In [4]:
import torch
import os
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import AdamW
from datasets import Dataset
import torch.nn.functional as F
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare 5 QA samples for training and validation
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = tokenizer.eos_token_id  # Ignore question and padding tokens

    inputs["labels"] = labels
    return inputs

# Convert samples to dataset and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# Convert to PyTorch DataLoader
batch_size = 2

def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch])
    attention_mask = torch.tensor([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["labels"] for item in batch])
    return input_ids, attention_mask, labels

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Cross-entropy loss function ignoring padding tokens
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model based on validation loss
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./gpt2-qa-best-loss-cml"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training and evaluation functions
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

        # Save best model based on validation loss
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10)

# Load the best fine-tuned model for inference
model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Epoch 1/10, Training Loss: 4.3919, Validation Loss: 1.5485
Best model saved at epoch 1 with validation loss: 1.5485
Epoch 2/10, Training Loss: 0.9726, Validation Loss: 0.9626
Best model saved at epoch 2 with validation loss: 0.9626
Epoch 3/10, Training Loss: 0.6966, Validation Loss: 0.6646
Best model saved at epoch 3 with validation loss: 0.6646
Epoch 4/10, Training Loss: 0.4566, Validation Loss: 0.4742
Best model saved at epoch 4 with validation loss: 0.4742
Epoch 5/10, Training Loss: 0.2667, Validation Loss: 0.4040
Best model saved at epoch 5 with validation loss: 0.4040
Epoch 6/10, Training Loss: 0.2041, Validation Loss: 0.3616
Best model saved at epoch 6 with validation loss: 0.3616
Epoch 7/10, Training Loss: 0.1894, Validation Loss: 0.3174
Best model saved at epoch 7 with validation loss: 0.3174
Epoch 8/10, Training Loss: 0.1186, Validation Loss: 0.2593
Best model saved at epoch 8 with validation loss: 0.2593
Epoch 9/10, Training Loss: 0.1020, Validation Loss: 0.2046
Best model sa

#Naive Entropy (Token wise Entropy)

In [6]:
import torch
import os
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import Dataset
import torch.nn.functional as F
import random
import numpy as np

model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Inference function with entropy-based uncertainty calculation
def generate_answer_with_entropy(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)

    # Generate with output scores to get logits at each generation step
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            output_scores=True,
            return_dict_in_generate=True
        )

    # Calculate entropy for each generated token (from the output scores)
    entropies = []
    for score in output.scores:
        # score shape: [batch_size, vocab_size]
        probs = F.softmax(score, dim=-1)
        entropy = -torch.sum(probs * torch.log(probs + 1e-10), dim=-1)  # entropy for each sample in the batch
        entropies.append(entropy)
    # Average entropy over generated tokens
    mean_entropy = torch.mean(torch.stack(entropies))

    # Decode generated text and extract answer part
    answer = tokenizer.decode(output.sequences[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer, mean_entropy.item()

# Example inference
question = "What is the capital of France?"
answer, uncertainty = generate_answer_with_entropy(question)
print(f"Q: {question}\nA: {answer}\nMean Entropy (Uncertainty): {uncertainty:.4f}")


Q: What is the capital of France?
A: The capital of France is Paris.
Mean Entropy (Uncertainty): 0.7248


#Variant-1:Semantic Entropy using another embedding model to obtain the embedding

In [7]:
!pip -q install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.5 MB/s eta 0:00:00


In [8]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# Generate top-k diverse answers
def generate_topk_answers(question, top_k=20):
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)

    answers = []
    for _ in range(top_k):
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,           # enable sampling
            top_k=top_k,              # restrict sampling to top_k tokens
            temperature=1.0,          # control randomness
        )
        answer_text = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
        answers.append(answer_text)
    return answers

# Compute semantic entropy
def calculate_semantic_entropy(answers):
    embeddings = embedding_model.encode(answers, convert_to_tensor=True)

    # Compute cosine similarity matrix
    similarity_matrix = F.cosine_similarity(
        embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=-1
    ).cpu().numpy()

    # Convert similarities to probabilities (normalize similarities)
    similarity_sums = similarity_matrix.sum(axis=1, keepdims=True)
    semantic_probs = similarity_sums / similarity_sums.sum()

    # Compute entropy from semantic probabilities
    semantic_entropy = -np.sum(semantic_probs * np.log(semantic_probs + 1e-10))

    return semantic_entropy

# Example function integrating all
def generate_answer_with_semantic_entropy(question, top_k=20):
    answers = generate_topk_answers(question, top_k=top_k)
    semantic_entropy = calculate_semantic_entropy(answers)

    # Most frequent generated answer (simple consensus method)
    unique_answers, counts = np.unique(answers, return_counts=True)
    consensus_answer = unique_answers[np.argmax(counts)]

    return consensus_answer, semantic_entropy, answers

# Example usage:
question = "What is the capital of France?"
consensus_answer, semantic_entropy, all_answers = generate_answer_with_semantic_entropy(question, top_k=20)

print(f"Question: {question}")
print(f"Consensus Answer: {consensus_answer}")
print(f"Semantic Entropy: {semantic_entropy:.4f}")
print("\nTop-20 Generated Answers:")
for idx, ans in enumerate(all_answers, 1):
    print(f"{idx}: {ans}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Question: What is the capital of France?
Consensus Answer: France is Paris.
Semantic Entropy: 2.9518

Top-20 Generated Answers:
1: The capital Louis XVI is Paris.
2: Paris.
3: Louis XVI is the capital of the French capital.
4: Paris is Paris .
5: Paris is Paris, France.
6: France is Paris.
7: Capital is Paris.
8: France is Paris
9: A capital is Paris.
10: Louis XVI.
11: The capital of France is Paris.
12: The capital of France is Paris.
13: 
14: France is Paris.
15: La France is Paris.
16: Paris is Paris.
17: Louis XIV
18: Paris was founded on the land of Paris, the land of Paris is France.
19: France is Paris.
20: The Capital of France is Paris, France.).


#Variant-2: Semantic Entropy using another entailment or NLI model to obtain the embedding

NLI Model bigger/slower: cross-encoder/nli-deberta-v3-large

In [13]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import CrossEncoder
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load GPT-2 QA Model
model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load NLI entailment model
nli_model = CrossEncoder("cross-encoder/nli-deberta-v3-large", device=device)

# Generate top-k diverse answers using GPT-2
def generate_topk_answers(question, top_k=20):
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)

    answers = []
    for _ in range(top_k):
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=top_k,
            temperature=1.0,
        )
        answer_text = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
        answers.append(answer_text)
    return answers

# Compute semantic entropy using entailment model
def calculate_entailment_entropy(question, answers):
    # Construct premise-hypothesis pairs for entailment scoring
    pairs = [(question, ans) for ans in answers]

    # Obtain entailment scores ([contradiction, neutral, entailment])
    entailment_logits = nli_model.predict(pairs, convert_to_numpy=True)
    entailment_probs = torch.softmax(torch.tensor(entailment_logits), dim=-1).numpy()

    # Use entailment probabilities (3rd column: entailment) to calculate entropy
    entailment_scores = entailment_probs[:, 2]  # Probability of entailment
    entailment_distribution = entailment_scores / entailment_scores.sum()

    semantic_entropy = -np.sum(entailment_distribution * np.log(entailment_distribution + 1e-10))

    return semantic_entropy

# Integrated generation and entropy computation
def generate_answer_with_entailment_entropy(question, top_k=20):
    answers = generate_topk_answers(question, top_k=top_k)
    semantic_entropy = calculate_entailment_entropy(question, answers)

    # Consensus answer (most frequent)
    unique_answers, counts = np.unique(answers, return_counts=True)
    consensus_answer = unique_answers[np.argmax(counts)]

    return consensus_answer, semantic_entropy, answers

# Example usage:
question = "What is the capital of France?"
consensus_answer, semantic_entropy, all_answers = generate_answer_with_entailment_entropy(question, top_k=20)

print(f"Question: {question}")
print(f"Consensus Answer: {consensus_answer}")
print(f"Semantic Entropy (Entailment-based): {semantic_entropy:.4f}")
print("\nTop-20 Generated Answers:")
for idx, ans in enumerate(all_answers, 1):
    print(f"{idx}: {ans}")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Question: What is the capital of France?
Consensus Answer: France is Paris.
Semantic Entropy (Entailment-based): 2.8972

Top-20 Generated Answers:
1: Paris is Paris, the capital of France.
2: Paris is Paris.
3: France is Paris.
4: France is Paris, France.
5: The capital of France is Paris.
6: The Capital of France is Paris.
7: Paris is the capital of France.
8: The capital of France is Paris.)
9: The capital France is Paris France.
10: The capital of France is Paris.
11: An Capital is France.
12: France is Paris.
13: France is Paris, capital of France.
14: France is located in Paris, France.
15: The capital of France is Paris and it is located at Paris, France.
16: The capital of France is Paris.
17: Paris is Paris, France.
18: The capital of France is Paris, Paris).
19: France is Paris.
20: The capital of Paris is Paris, Paris.


Variant-3: NLI Model lighter/faster: cross-encoder/nli-MiniLM2-L6-H768

In [14]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import CrossEncoder
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load GPT-2 QA Model
model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load lightweight NLI entailment model
nli_model = CrossEncoder("cross-encoder/nli-MiniLM2-L6-H768", device=device)

# Generate top-k diverse answers using GPT-2
def generate_topk_answers(question, top_k=20):
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)

    answers = []
    for _ in range(top_k):
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=top_k,
            temperature=1.0,
        )
        answer_text = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
        answers.append(answer_text)
    return answers

# Compute semantic entropy using entailment model
def calculate_entailment_entropy(question, answers):
    # Construct premise-hypothesis pairs for entailment scoring
    pairs = [(question, ans) for ans in answers]

    # Obtain entailment scores ([contradiction, neutral, entailment])
    entailment_logits = nli_model.predict(pairs, convert_to_numpy=True)
    entailment_probs = torch.softmax(torch.tensor(entailment_logits), dim=-1).numpy()

    # Use entailment probabilities (3rd column: entailment) to calculate entropy
    entailment_scores = entailment_probs[:, 2]  # Probability of entailment
    entailment_distribution = entailment_scores / entailment_scores.sum()

    semantic_entropy = -np.sum(entailment_distribution * np.log(entailment_distribution + 1e-10))

    return semantic_entropy

# Integrated generation and entropy computation
def generate_answer_with_entailment_entropy(question, top_k=20):
    answers = generate_topk_answers(question, top_k=top_k)
    semantic_entropy = calculate_entailment_entropy(question, answers)

    # Consensus answer (most frequent)
    unique_answers, counts = np.unique(answers, return_counts=True)
    consensus_answer = unique_answers[np.argmax(counts)]

    return consensus_answer, semantic_entropy, answers

# Example usage:
question = "What is the capital of France?"
consensus_answer, semantic_entropy, all_answers = generate_answer_with_entailment_entropy(question, top_k=20)

print(f"Question: {question}")
print(f"Consensus Answer: {consensus_answer}")
print(f"Semantic Entropy (Entailment-based): {semantic_entropy:.4f}")
print("\nTop-20 Generated Answers:")
for idx, ans in enumerate(all_answers, 1):
    print(f"{idx}: {ans}")

config.json:   0%|          | 0.00/875 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Question: What is the capital of France?
Consensus Answer: The capital of France is Paris.
Semantic Entropy (Entailment-based): 2.9879

Top-20 Generated Answers:
1: The capital of France is Paris).
2: The capital is Paris and France.
3: In Paris, the capital of Paris is the capital of Paris (French).
4: The capital of France is Paris.
5: France is Saint-Louis.
6: Paris.
7: Capital of Paris is Paris .
8: France is Paris.
9: Paris is Paris (Paris)
10: Paris is Paris.
11: It is Paris.
12: The capital of France is Paris (Paris), France).
13: The Capital de France is Paris."
14: Paris is Paris de France.
15: the capital of France is Paris.
16: The Capital of France is Paris, France.
17: The capital of France is Paris, Paris, France.
18: The capital of France is Paris.
19: The capital of France is Paris.
20: The capital of Paris is Paris.


#Variant-4: Semantic Entropy using same LLM to obtain the embedding

In [17]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sklearn.cluster import KMeans
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load GPT-2 QA Model
model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name, output_hidden_states=True).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Generate top-k diverse answers using GPT-2 and obtain embeddings
def generate_topk_answers(question, top_k=20):
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)

    answers = []
    embeddings = []
    for _ in range(top_k):
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=top_k,
            temperature=1.0,
            return_dict_in_generate=True
        )
        answer_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True).split("Answer:")[-1].strip()
        answers.append(answer_text)

        # Get embedding by encoding generated answer separately
        encoded_answer = tokenizer(answer_text, return_tensors="pt", truncation=True, max_length=50).to(device)
        with torch.no_grad():
            answer_output = model(**encoded_answer, output_hidden_states=True)
        answer_embedding = answer_output.hidden_states[-1][0, -1, :].cpu().numpy()
        embeddings.append(answer_embedding)

    return answers, np.array(embeddings)

# Compute semantic entropy using clustering of GPT-2 embeddings
def calculate_semantic_entropy(embeddings, num_clusters=5):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    labels = kmeans.fit_predict(embeddings)

    # Compute probabilities of each cluster
    label_counts = np.bincount(labels)
    label_probs = label_counts / label_counts.sum()

    # Calculate entropy
    semantic_entropy = -np.sum(label_probs * np.log(label_probs + 1e-10))

    return semantic_entropy

# Integrated generation and entropy computation
def generate_answer_with_semantic_entropy(question, top_k=20, num_clusters=5):
    answers, embeddings = generate_topk_answers(question, top_k=top_k)
    semantic_entropy = calculate_semantic_entropy(embeddings, num_clusters=num_clusters)

    # Consensus answer (most frequent)
    unique_answers, counts = np.unique(answers, return_counts=True)
    consensus_answer = unique_answers[np.argmax(counts)]

    return consensus_answer, semantic_entropy, answers

# Example usage:
question = "What is the capital of France?"
consensus_answer, semantic_entropy, all_answers = generate_answer_with_semantic_entropy(question, top_k=20, num_clusters=5)

print(f"Question: {question}")
print(f"Consensus Answer: {consensus_answer}")
print(f"Semantic Entropy (GPT-2 embeddings clustering-based): {semantic_entropy:.4f}")
print("\nTop-20 Generated Answers:")
for idx, ans in enumerate(all_answers, 1):
    print(f"{idx}: {ans}")


The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Question: What is the capital of France?
Consensus Answer: The capital of France is Paris.
Semantic Entropy (GPT-2 embeddings clustering-based): 1.5048

Top-20 Generated Answers:
1: Paris.
2: Paris is the capital of the Paris capital.
3: Paris is the capital of France.
4: The Capital of France is La France.
5: Paris is the capital of France.
6: The capital of France is Paris.
7: France is French.
8: Paris is Paris Capital.
9: Paris is Paris.)
10: The capital is Paris.
11: The capital of France is Paris, France.
12: Paris is Paris.
13: France is France.
14: The capital of France consists of Paris, Paris, a Roman city founded in 1490, Paris is France; and Paris is the capital of the Roman Catholic Church.
15: Paris is Paris, located near Porte de Ville.
16: Paris.
17: The capital of France is Paris.
18: The capital of France is Paris.
19: The capital of France is Paris.
20: The capital of France is Paris.
